# End to End Demo of Speech-2-PR AI

Install packages in `requirements.txt`

### Helper

In [1]:
import os
import sys
import importlib.util
import ast
import re
import librosa
import torch
import qdrant_client
from pathlib import Path
from openai import OpenAI
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

c:\Users\srava\Anaconda3\envs\speech2code\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from github import Github
from github.GithubException import GithubException

In [3]:
os.environ["openaikey"] = input("Please enter your Open AI API key")

In [4]:
class OpenAILLM:
    def __init__(self, model_name="gpt-4o", max_tokens=1000):
        self.openai_client = OpenAI(
            api_key=os.getenv("openaikey"))
        assert(model_name in [model.id for model in self.openai_client.models.list().data])
        self.model_name = model_name
        self.max_tokens = max_tokens
    
    def call_llm(self, prompt):
        text_list = [{"type": "text", "text": prompt}]
        try:
            response = self.openai_client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {
                        "role": "user",
                        "content": text_list,
                    }
                ],
                max_tokens=self.max_tokens,
            )
        except Exception as e:
            print(e)
            return "LLM Failed, please try again"
        return response.choices[0].message.content
    
    def __call__(self, prompt):
        return  self.call_llm(prompt)

In [5]:
def getLlm(max_tokens=1000):
    return OpenAILLM(max_tokens=max_tokens)

def runLlm(llm, formattedPrompt):
   return llm(formattedPrompt)

In [6]:
llm = getLlm()
runLlm(llm,"Hello")

'Hello! How can I help you today?'

## Part A: Upload your audio file (meeting audio)

In [7]:
# Load your audio file using librosa
def load_audio(file_path, target_sample_rate=16000):
    audio, sample_rate = librosa.load(file_path, sr=None)
    if sample_rate != target_sample_rate:
        audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=target_sample_rate)
    return audio, target_sample_rate

In [9]:
# Load your audio file (replace 'your_audio_file.wav' with the path to your audio file)
audio_file_path = 'DesignDiscussion.wav'
audio, sample_rate = load_audio(audio_file_path)

## Part B: Use ASR (Whisper) to transcribe meeting audio

In [10]:
# Load model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-small"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype,
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

c:\Users\srava\Anaconda3\envs\speech2code\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Convert waveform to numpy array and create a sample dictionary
sample = {
    "array": audio,
    "sampling_rate": sample_rate
}

result = pipe(sample)
transcript_text = result["text"]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [12]:
print(transcript_text)

 Hi all, thanks for joining. Today we're going to discuss the feature revamp for the multimodal RAG application to improve user experience. First on our list is adding a chat functionality to the application so that users can maintain chat history and ask questions over some of the previous interactions they've had with the application. In order to work successfully we need to ensure there is a chat reset, a chat delete, and a chat modal that keeps track of all the different chats our user is having with their application in context.


## Part C: Build a Code Spec

In [13]:
prompt = f'''
### Instructions for Extracting Information for a Code Spec

1. **Objective Statement:**
   - Identify and extract statements that describe the overall goal or purpose of the feature being discussed. Look for phrases like "the purpose of this feature is..." or "the main objective is...".

2. **Requirements:**
   - **Functional Requirements:**
     - Extract details that describe what the system should do. Look for specific functionalities, behaviors, or actions that the feature must perform. Key phrases might include "the system must...", "the feature should allow...", "it needs to...", etc.
   - **Non-functional Requirements:**
     - Identify requirements that describe how the system performs certain functions. These might include performance metrics, usability standards, reliability, and other quality attributes. Look for phrases like "it should be fast...", "it must be secure...", "it needs to handle up to X users...", etc.

3. **Use Case/User Story:**
   - Extract information that details how a user will interact with the feature. Look for narratives or scenarios that describe the user’s actions and interactions with the system. Phrases might include "as a user, I want to...", "the user will...", "in this scenario...".

4. **Deliverables:**
   - **Design New File:**
     - Identify references to creating new files or components as part of the feature implementation. Look for phrases like "we need to create a new...", "design a new...", etc.
   - **Modify Existing File:**
     - Extract details about changes to existing files or components. Look for phrases like "we will modify...", "update the existing...", etc.
   - **Documentation:**
     - Identify information about required documentation. Look for mentions of writing, documenting, or adding comments in the repository. Phrases might include "document the...", "write the docs...", "add comments to...".

---

### Example Analysis

**Objective Statement:**
- Extract: "The main objective of this feature is to provide users with a seamless experience when accessing their profile."

**Requirements:**
- **Functional:**
  - Extract: "The system must allow users to update their profile information."
  - Extract: "The feature should enable users to upload profile pictures."
- **Non-functional:**
  - Extract: "The profile update process should be completed within 2 seconds."
  - Extract: "The system must be able to handle 10,000 concurrent profile updates."

**Use Case/User Story:**
- Extract: "As a user, I want to be able to change my profile picture so that I can personalize my account."

**Deliverables:**
- **Design New File:**
  - Extract: "We need to design a new profile picture upload module."
- **Modify Existing File:**
  - Extract: "We will update the user model to include the new profile picture attribute."
- **Documentation:**
  - Extract: "Document the new API endpoints for profile updates in the repository."

---
Use the structure above to generate a code spec from the following meeting transcript. The output will be a summary that does not have to include timestamp nor keep the original sentence structure. Keep the number of bullet points minimal while keeping essential information.

---
{transcript_text}

'''

In [14]:
code_spec = runLlm(llm, prompt)

In [15]:
print(code_spec)

### Code Spec for Multimodal RAG Application Feature Revamp

**Objective Statement:**
- The main objective of this feature is to provide an enhanced user experience by adding chat functionality to the multimodal RAG application.

**Requirements:**
- **Functional:**
  - The system must allow users to maintain chat history.
  - The feature should enable users to ask questions based on their previous interactions.
  - The system must include functionalities for chat reset and chat delete.
- **Non-functional:**
  - N/A (not specified in the transcript).

**Use Case/User Story:**
- As a user, I want to maintain and interact with chat history to ask follow-up questions based on my previous interactions with the application.

**Deliverables:**
- **Design New File:**
  - Design a new chat modal to keep track of all the different chats in context.
- **Modify Existing File:**
  - N/A (not specified in the transcript).
- **Documentation:**
  - Document the new chat functionalities including chat 

## Part D: Retrieve relevant files from RAG

### Clone Repository

In [16]:
!git clone "git@github.com:SVJayanthi/Multimodal-RAG.git"

Cloning into 'Multimodal-RAG'...


### Setup Retriever

In [18]:
# For each file in a directory, get source code & metadata about file_name / dir
## Ingest .py files

def get_file_metadata_and_content(directory):
    supported_extensions = ('.py')
    files_data = []

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(supported_extensions):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    content = f.read()
                
                file_data = {
                    'file_name': file,
                    'file_path': file_path,
                    'content': content
                }
                files_data.append(file_data)
    
    return files_data

# Example usage
directory_path = 'Multimodal-RAG'
files_metadata_and_content = get_file_metadata_and_content(directory_path)

In [19]:
len(files_metadata_and_content)

22

In [23]:
collection_name = "code_collection"
def setup_retriever(corpus_data, embed_ids=True):
    q_client = qdrant_client.QdrantClient(":memory:")
    if embed_ids:
        corpus_texts = ["File Name: "+i['file_name'] + "\n " + i['content'] for i in corpus_data]
    else:
        corpus_texts = [i['text'] for i in corpus_data]
    
    q_client.add(
        collection_name,
        documents = corpus_texts,
        metadata=corpus_data,
    )

    return q_client

In [26]:
retriever = setup_retriever(files_metadata_and_content)

100%|██████████| 77.7M/77.7M [00:01<00:00, 63.9MiB/s]


### Code Search

In [27]:
def retrieve(retriever, question, top_k=3):
    search_result = retriever.query(
        collection_name=collection_name,
        query_text=question,
        limit=top_k
    )

    return search_result

In [39]:
results = retrieve(retriever, transcript_text, 10)
file_paths = [r.metadata['file_path'] for r in results]
selected_file_path = file_paths[0]
selected_file_path

'Multimodal-RAG\\chatapp\\chatapp\\chatapp.py'

## Part E: Build Context for File

In [40]:
def extract_imports(file_path):
    with open(file_path, 'r') as file:
        tree = ast.parse(file.read(), filename=file_path)
    imports = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imports.append(alias.name)
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                imports.append(node.module)
    return imports

In [49]:
def is_local_module(module_name, repo_path):
    module_path = os.path.join(repo_path, *module_name.split('.'))
    return (os.path.isfile(f"{module_path}.py") or os.path.isdir(module_path))

def get_local_imports(imports, repo_path):
    local_imports = []
    for imp in imports:
        if is_local_module(imp, repo_path):
            local_imports.append(imp)
    return local_imports


In [46]:
get_local_imports(extract_imports(selected_file_path),  )

Multimodal-RAG\chatapp\reflex
Multimodal-RAG\chatapp\chatapp\components
Multimodal-RAG\chatapp\chatapp\state
Multimodal-RAG\chatapp\chatapp


['chatapp.components', 'chatapp.state', 'chatapp']

In [47]:
def gather_code_and_imports(file_path, repo_path, gathered_files=None):
    if gathered_files is None:
        gathered_files = set()

    if file_path in gathered_files:
        return ""
    
    gathered_files.add(file_path)

    with open(file_path, 'r') as file:
        code = file.read()

    imports = extract_imports(file_path)
    local_imports = get_local_imports(imports, repo_path)

    for local_module in local_imports:
        local_file_path = os.path.join(repo_path, *local_module.split('.')) + ".py"
        if os.path.isfile(local_file_path):
            code += gather_code_and_imports(local_file_path, repo_path, gathered_files)
    
    return code

def get_all_code(main_file_path, repo_path):
    return gather_code_and_imports(main_file_path, repo_path)

In [50]:
repo_path = "Multimodal-RAG\chatapp"
combined_context = get_all_code(selected_file_path, repo_path)

In [78]:
# Gathers code and imports from a given file, annotating with filenames and tracking hierarchy
def gather_code_and_imports(file_path, repo_path, gathered_files=None, file_hierarchy=None):
    if gathered_files is None:
        gathered_files = set()
    if file_hierarchy is None:
        file_hierarchy = {}

    if file_path in gathered_files:
        return "", file_hierarchy
    
    gathered_files.add(file_path)

    with open(file_path, 'r') as file:
        code = file.read()
    
    annotated_code = f"# Begin of {file_path}\n{code}\n# End of {file_path}\n"
    
    imports = extract_imports(file_path)
    local_imports = get_local_imports(imports, repo_path)

    file_hierarchy[file_path] = []

    for local_module in local_imports:
        local_file_path = os.path.join(repo_path, *local_module.split('.')) + ".py"
        if os.path.isfile(local_file_path):
            file_hierarchy[file_path].append(local_file_path)
            sub_code, file_hierarchy = gather_code_and_imports(local_file_path, repo_path, gathered_files, file_hierarchy)
            annotated_code += sub_code
    
    return annotated_code, file_hierarchy

# Gets all code from the main file and its local imports, and provides the file hierarchy
def get_all_code(main_file_path, repo_path):
    code, file_hierarchy = gather_code_and_imports(main_file_path, repo_path)
    return code, file_hierarchy

# Displays the file hierarchy in a readable format
def display_file_hierarchy(file_hierarchy, indent=0):
    f_string = ""
    for file, imports in file_hierarchy.items():
        f_string += (' ' * indent + os.path.basename(file)) + "\n"
        if imports:
            f_string += "imports from " + display_file_hierarchy({imp: file_hierarchy[imp] for imp in imports}, indent + 4)
    return f_string


In [73]:
repo_path = "Multimodal-RAG\chatapp"
combined_context, file_hierarchy = get_all_code(selected_file_path, repo_path)

## Part F: Perform Code Change(s) using Code Spec & Context about File

In [80]:
prompt_to_llm = (
    f"Read the objective statement and write the changes in the {selected_file_path} file based on the instructions and details provided.\n"
    + str(code_spec)
    + "Here is the hierarchy \n: "
    + display_file_hierarchy(file_hierarchy)
    + "Here is the source codes for all the files mentioned above in the hierarchy. \n"
    + str(combined_context)
)

In [81]:
llm_output = runLlm(llm, prompt_to_llm)

In [82]:
print(llm_output)

Based on the objective statement and feature requirements, you will need to make changes to the `Multimodal-RAG\chatapp\chatapp\chatapp.py` file to enhance the user experience by adding new chat functionalities, including maintaining chat history, enabling users to ask questions based on previous interactions, and managing chat resets and deletes.

Here's how you can modify the `chatapp.py` to achieve the stated requirements:

### Changes in `Multimodal-RAG\chatapp\chatapp\chatapp.py`:

1. **Import necessary components for chat reset and delete functionalities:**
2. **Include chat functionalities in the navigation bar:**
3. **Include state management for chat operations:**

### Modified `Multimodal-RAG\chatapp\chatapp\chatapp.py`:

```python
"""The main Chat app."""

import reflex as rx
from chatapp.components import chat, navbar

# Import State for managing current and new states
from chatapp.state import State 
from chatapp import style

def index() -> rx.Component:
    """The main a

In [83]:
def extract_python_code(text):
    # Define the regular expression pattern
    pattern = r'```python(.*?)```'
    
    # Use re.findall to extract all occurrences
    matches = re.findall(pattern, text, re.DOTALL)
    
    return matches

In [84]:
code_update = extract_python_code(llm_output)[0]

In [85]:
print(code_update)


"""The main Chat app."""

import reflex as rx
from chatapp.components import chat, navbar

# Import State for managing current and new states
from chatapp.state import State 
from chatapp import style

def index() -> rx.Component:
    """The main app."""
    
    return rx.chakra.vstack(
        navbar(),
        chat.chat(),
        chat.action_bar(),
        background_color=rx.color("mauve", 1),
        color=rx.color("mauve", 12),
        min_height="100vh",
        align_items="stretch",
        spacing="0",
    )

def chat_management() -> rx.Component:
    """Component for managing chat functionalities."""
    
    return rx.chakra.hstack(
        rx.chakra.button(
            "Reset Chat",
            on_click=State.reset_chat
        ),
        rx.chakra.button(
            "Delete Chat",
            on_click=State.delete_chat
        )
    )

# Add state and page to the app.
app = rx.App(
    theme=rx.theme(
        appearance="dark",
        accent_color="violet",
    ),
)



## Part G: Perform a PR to Github

In [86]:
# Authentication to Github
GITHUB_TOKEN = input("Please enter your GitHub token: ")

In [121]:
# Fill out the reqs to make a PR
REPO_NAME = 'Multimodal-RAG'
NEW_BRANCH_NAME = 'speech2code'
BASE_BRANCH = 'develop'
FILE_PATH = selected_file_path
COMMIT_MESSAGE = 'Test Commit'
PR_TITLE = 'Test Hackthon'
PR_BODY = 'Adding a comment'

In [122]:
new_content_str = code_update

In [ ]:
try:
    g = Github(GITHUB_TOKEN)
    # Get the repository
    repo = g.get_repo(REPO_NAME)
    base_branch_ref = repo.get_git_ref(f"heads/{BASE_BRANCH}")
    repo.create_git_ref(ref=f"refs/heads/{NEW_BRANCH_NAME}", sha=base_branch_ref.object.sha)
    print(f"Branch '{NEW_BRANCH_NAME}' created successfully.")

    # Get the file content
    file_content = repo.get_contents(FILE_PATH, ref=NEW_BRANCH_NAME)
    # Decode the content to a string
    content_str = file_content.decoded_content.decode('utf-8')

    # Modify the file content (example modification: append a line)
    new_content_str = content_str + code_update

    # Update the file with new content
    repo.update_file(
        path=FILE_PATH,
        message="Modify existing file in new branch",
        content=new_content_str,
        sha=file_content.sha,
        branch=NEW_BRANCH_NAME
    )
    print(f"File '{FILE_PATH}' modified successfully in branch '{NEW_BRANCH_NAME}'.")

    # Create the pull request
    pr = repo.create_pull(
        title=f"Modify existing file {Path(selected_file_path).stem} in new branch",
        body="This pull request modifies an existing file in the new branch.",
        head=NEW_BRANCH_NAME,
        base=BASE_BRANCH
    )

    print(f"Pull Request created: {pr.html_url}")

except GithubException as e:
    print(f"Failed to create branch or pull request: {e.data}")
except Exception as e:
    print(f"An error occurred: {e}")